In [2]:
! pip install mne datasets evaluate

In [3]:
!pip install numpy==1.23.0

In [4]:
import mne
import random
import numpy as np
import pandas as pd
import keras
import tensorflow as tf
import os
import evaluate
import datasets
from datasets import load_dataset
from huggingface_hub import hf_hub_download, login

In [5]:
login()

In [6]:
hf_hub_download(repo_id="groshanr/EEG2Text-Raw", filename="Murphyetal.zip", repo_type="dataset")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


'/root/.cache/huggingface/hub/datasets--groshanr--EEG2Text-Raw/snapshots/2d4b1ce038376e84109a5bbe8d24df6377f439c8/Murphyetal.zip'

In [7]:
%cd /root/.cache/huggingface/hub/datasets--groshanr--EEG2Text-Raw/snapshots/2d4b1ce038376e84109a5bbe8d24df6377f439c8

/root/.cache/huggingface/hub/datasets--groshanr--EEG2Text-Raw/snapshots/2d4b1ce038376e84109a5bbe8d24df6377f439c8


In [8]:
!unzip Murphyetal.zip

Archive:  Murphyetal.zip
replace Murphyetal/rsvp_session4_files31-44-epo.fif? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [9]:
directory = '/root/.cache/huggingface/hub/datasets--groshanr--EEG2Text-Raw/snapshots/2d4b1ce038376e84109a5bbe8d24df6377f439c8/Murphyetal'
epoch_files = []

for root, dirs, files in os.walk(directory):
	for filename in files:
		if filename.endswith('.fif'):
				epoch_files.append(os.path.join(root, filename))

In [10]:
subsample = random.sample(epoch_files, 1)

In [11]:
epoch_list = []
for f in subsample:
    epoch_list.append(mne.read_epochs(f))

Reading /root/.cache/huggingface/hub/datasets--groshanr--EEG2Text-Raw/snapshots/2d4b1ce038376e84109a5bbe8d24df6377f439c8/Murphyetal/rsvp_session1_files141-151-epo.fif ...
    Read a total of 1 projection items:
        Average EEG reference (1 x 64) active
    Found the data of interest:
        t =    -200.00 ...     900.00 ms
        0 CTF compensation matrices available
Adding metadata with 16 columns
2790 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 1)
1 projection items activated


In [12]:
epochs = mne.concatenate_epochs(epoch_list)

Adding metadata with 16 columns
2790 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 1)


In [13]:
epochs_df = epochs.to_data_frame()

In [14]:
metadata = epochs.metadata

In [17]:
metadata['pos'] = metadata['pos'].astype('category')
metadata['pos'] = metadata['pos'].cat.codes

In [30]:
import keras
from keras import layers

CNN_input = keras.Input(shape=(32, 9, 11, 276, 1), batch_size=32, name = 'SentenceInput')

#Conv Block 1
x = layers.TimeDistributed(layers.BatchNormalization(name = 'BatchNorm1'))(CNN_input)
x = layers.TimeDistributed(layers.Conv3D(16, kernel_size=(5,5, 1), strides=1, padding = 'same', activation='relu', name = 'Conv1'))(x)

#Find time attention for each channel
time_dense1 = layers.TimeDistributed(layers.Dense(32, activation = 'relu', name = 'TimeDense1_1'))
time_dense2 = layers.TimeDistributed(layers.Dense(16, activation = 'relu', name = 'TimeDense1_2'))

fx_max = layers.TimeDistributed(layers.MaxPooling3D(pool_size = (9, 11, 1), name = 'TimeMaxPool1'))(x)
fx_max = time_dense1(fx_max)
fx_max = time_dense2(fx_max)

fx_avg = layers.TimeDistributed(layers.AveragePooling3D(pool_size= (9, 11, 1), name = 'TimeAvgPool1'))(x)
fx_avg = time_dense1(fx_avg)
fx_avg = time_dense2(fx_avg)

time_attention = layers.Add(name = 'TimeAdd1')([fx_avg, fx_max])
time_attention = layers.Activation('sigmoid', name = 'TimeSigmoid1')(time_attention)

fx = layers.Multiply(name = 'TimeAttnCreate1')([time_attention, x])

#Find space attention for each channel
fx_max_space = layers.TimeDistributed(layers.MaxPooling3D(pool_size = (1,1,276), name = 'SpaceMaxPool1'))(fx)
fx_avg_space = layers.TimeDistributed(layers.AveragePooling3D(pool_size = (1,1,276), name = 'SpaceAvgPool1'))(fx)
space_attention = layers.Concatenate(axis = 3, name = 'SpaceConcat1')([fx_avg_space, fx_max_space])
space_attention = layers.TimeDistributed(layers.Conv3D(filters = 1, kernel_size = (2,2,1), strides = (1,2,1), padding= 'same', activation='sigmoid', name = 'SpaceConvAttn1'))(space_attention)

fx = layers.Multiply(name = 'SpaceAttnCreate1')([space_attention, fx])

#Apply Time and Space attention
block1out = layers.Add(name = 'AttnApply1')([fx, x])

#Conv Block 2

x = layers.TimeDistributed(layers.BatchNormalization(name = 'BatchNorm2'))(block1out)
x = layers.TimeDistributed(layers.Conv3D(32, kernel_size=(5,5, 1), strides=1, padding = 'same', activation='relu', name = 'Conv2'))(x)

#Find time attention for each channel
time_dense3 = layers.TimeDistributed(layers.Dense(64, activation = 'relu', name = 'TimeDense2_1'))
time_dense4 = layers.TimeDistributed(layers.Dense(32, activation = 'relu', name = 'TimeDense2_2'))

fx_max = layers.TimeDistributed(layers.MaxPooling3D(pool_size = (9, 11, 1), name = 'TimeMaxPool2'))(x)
fx_max = time_dense3(fx_max)
fx_max = time_dense4(fx_max)

fx_avg = layers.TimeDistributed(layers.AveragePooling3D(pool_size= (9, 11, 1), name = 'TimeAvgPool2'))(x)
fx_avg = time_dense3(fx_avg)
fx_avg = time_dense4(fx_avg)

time_attention = layers.Add(name = 'TimeAdd2')([fx_avg, fx_max])
time_attention = layers.Activation('sigmoid', name = 'TimeSigmoid2')(time_attention)

fx = layers.Multiply(name = 'TimeAttnCreate2')([time_attention, x])

#Find space attention for each channel
fx_max_space = layers.TimeDistributed(layers.MaxPooling3D(pool_size = (1,1,276), name = 'SpaceMaxPool2'))(fx)
fx_avg_space = layers.TimeDistributed(layers.AveragePooling3D(pool_size = (1,1,276), name = 'SpaceAvgPool2'))(fx)
space_attention = layers.Concatenate(axis = 3, name = 'SpaceConcat2')([fx_avg_space, fx_max_space])
space_attention = layers.TimeDistributed(layers.Conv3D(filters = 1, kernel_size = (2,2,2), strides = (1,2,1), padding= 'same', activation='sigmoid', name = 'SpaceConvAttn2'))(space_attention)

fx = layers.Multiply(name = 'SpaceAttnCreate2')([space_attention, fx])
block2out = layers.Add(name = 'TimeSpaceAttnApply2')([fx, x])

#Conv Block 3
x = layers.TimeDistributed(layers.BatchNormalization(name = 'BatchNorm3'))(block2out)
x = layers.TimeDistributed(layers.Conv3D(64, kernel_size=(5,5, 1), strides=1, padding = 'same', activation='relu', name = 'Conv3'))(x)

time_dense5 = layers.TimeDistributed(layers.Dense(32, activation = 'relu', name = 'TimeDense3_1'))
time_dense6 = layers.TimeDistributed(layers.Dense(64, activation = 'relu', name = 'TimeDense3_2'))

fx_max = layers.TimeDistributed(layers.MaxPooling3D(pool_size = (9, 11, 1), name = 'TimeMaxPool3'))(x)
fx_max = time_dense5(fx_max)
fx_max = time_dense6(fx_max)

fx_avg = layers.TimeDistributed(layers.AveragePooling3D(pool_size= (9, 11, 1), name = 'TimeAvgPool3'))(x)
fx_avg = time_dense5(fx_avg)
fx_avg = time_dense6(fx_avg)

time_attention = layers.Add(name = 'TimeAdd3')([fx_avg, fx_max])
time_attention = layers.Activation('sigmoid', name = 'TimeSigmoid3')(time_attention)

fx = layers.Multiply(name = 'TimeAttnCreate3')([time_attention, x])

#Find space attention for each channel
fx_max_space = layers.TimeDistributed(layers.MaxPooling3D(pool_size = (1,1,276), name = 'SpaceMaxPool3'))(fx)
fx_avg_space = layers.TimeDistributed(layers.AveragePooling3D(pool_size = (1,1,276), name = 'SapceAvgPool3'))(fx)
space_attention = layers.Concatenate(axis = 3, name = 'SpaceConcat3')([fx_avg_space, fx_max_space])
space_attention = layers.TimeDistributed(layers.Conv3D(filters = 1, kernel_size = (2,2,2), strides = (1,2,1), padding= 'same', activation='sigmoid', name = 'SpaceConv3'))(space_attention)

fx = layers.Multiply(name = 'SpaceAttnCreate3')([space_attention, fx])
block3out = layers.Add(name = 'TimeSpaceAttnApply3')([fx, x])

#Conv Block 4

x = layers.TimeDistributed(layers.BatchNormalization(name = 'BatchNorm4'))(block3out)
x = layers.TimeDistributed(layers.Conv3D(32, kernel_size=(5,5, 1), strides=1, padding = 'same', activation='relu', name = 'Conv4'))(x)

time_dense7 = layers.TimeDistributed(layers.Dense(64, activation = 'relu', name = 'TimeDense4_1'))
time_dense8 = layers.TimeDistributed(layers.Dense(32, activation = 'relu', name = 'TimeDense4_2'))

fx_max = layers.TimeDistributed(layers.MaxPooling3D(pool_size = (9, 11, 1), name='TimeMaxPool4'))(x)
fx_max = time_dense7(fx_max)
fx_max = time_dense8(fx_max)

fx_avg = layers.TimeDistributed(layers.AveragePooling3D(pool_size= (9, 11, 1), name = 'TimeAvgPool4'))(x)
fx_avg = time_dense7(fx_avg)
fx_avg = time_dense8(fx_avg)

time_attention = layers.Add(name = 'TimeAdd4')([fx_avg, fx_max])
time_attention = layers.Activation('sigmoid', name = 'TimeSigmoid4')(time_attention)

fx = layers.Multiply(name = 'TimeAttnCreate4')([time_attention, x])

#Find space attention for each channel
fx_max_space = layers.TimeDistributed(layers.MaxPooling3D(pool_size = (1,1,276), name = 'SpaceMaxPool4'))(fx)
fx_avg_space = layers.TimeDistributed(layers.AveragePooling3D(pool_size = (1,1,276), name = 'SpaceAvgPool4'))(fx)
space_attention = layers.Concatenate(axis = 3, name = 'SpaceConcat4')([fx_avg_space, fx_max_space])
space_attention = layers.TimeDistributed(layers.Conv3D(filters = 1, kernel_size = (2,2,2), strides = (1,2,1), padding= 'same', activation='sigmoid', name = 'SpaceConv4'))(space_attention)

fx = layers.Multiply(name = 'SpaceAttnCreate4')([space_attention, fx])
block4out = layers.Add(name = 'TimeSpaceAttnApply4')([fx, x])

# Final word vector maker
x = layers.TimeDistributed(layers.MaxPool3D(pool_size = (3,3,25), name = 'FinalMaxPool'))(block4out)
x = layers.TimeDistributed(layers.Flatten(name = 'Flatten'))(x)
x = layers.TimeDistributed(layers.Dense(64, activation='relu', name = 'FinalDense1'))(x)
x = layers.TimeDistributed(layers.Dense(32, activation='relu', name = 'FinalDense2'))(x)
x = layers.TimeDistributed(layers.Dense(64, activation='relu', name = 'FinalDense3'))(x)
x = layers.TimeDistributed(layers.Dense(16, name = 'PosOutput', activation='softmax'))(x)
CNN_output = layers.TimeDistributed(layers.Normalization(axis=-1, name = 'Pos Normalization'))(x)

pos_model = keras.Model(CNN_input, CNN_output)
pos_model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ SentenceInput             │ (32, 32, 9, 11, 276,   │              0 │ -                      │
│ (InputLayer)              │ 1)                     │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ time_distributed_86       │ (32, 32, 9, 11, 276,   │              4 │ SentenceInput[0][0]    │
│ (TimeDistributed)         │ 1)                     │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ time_distributed_87       │ (32, 32, 9, 11, 276,   │            416 │ time_distributed_86[0… │
│ (TimeDistributed)         │ 16)                    │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ time_distributed_91       │ (32, 32, 1, 1, 276,    │              0 │ time_distributed_87[0… │
│ (TimeDistributed)         │ 16)                    │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ time_distributed_90       │ (32, 32, 1, 1, 276,    │              0 │ time_distributed_87[0… │
│ (TimeDistributed)         │ 16)                    │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ time_distributed_88       │ (32, 32, 1, 1, 276,    │            544 │ time_distributed_90[0… │
│ (TimeDistributed)         │ 32)                    │                │ time_distributed_91[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ time_distributed_89       │ (32, 32, 1, 1, 276,    │            528 │ time_distributed_88[0… │
│ (TimeDistributed)         │ 16)                    │                │ time_distributed_88[1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ TimeAdd1 (Add)            │ (32, 32, 1, 1, 276,    │              0 │ time_distributed_89[1… │
│                           │ 16)                    │                │ time_distributed_89[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ TimeSigmoid1 (Activation) │ (32, 32, 1, 1, 276,    │              0 │ TimeAdd1[0][0]         │
│                           │ 16)                    │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ TimeAttnCreate1           │ (32, 32, 9, 11, 276,   │              0 │ TimeSigmoid1[0][0],    │
│ (Multiply)                │ 16)                    │                │ time_distributed_87[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ time_distributed_93       │ (32, 32, 9, 11, 1, 16) │              0 │ TimeAttnCreate1[0][0]  │
│ (TimeDistributed)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ time_distributed_92       │ (32, 32, 9, 11, 1, 16) │              0 │ TimeAttnCreate1[0][0]  │
│ (TimeDistributed)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ SpaceConcat1              │ (32, 32, 9, 22, 1, 16) │              0 │ time_distributed_93[0… │
│ (Concatenate)             │                        │                │ time_distributed_92[0… │
├──────────────────────

 Total params: 339,017 (1.29 MB)

 Trainable params: 338,758 (1.29 MB)

 Non-trainable params: 259 (1.02 KB)

In [31]:
pos_model.save('PosModel.keras')

In [32]:
empty = np.zeros(276)
sent_end = np.random.rand(9,11,276)
sent_empty = np.zeros((9,11,276))
pad = np.zeros(1)
special_characters = '!@#$%^&*()_+-=[],."\''

train_x = []
train_y =[]

for s in np.unique(metadata['sent_ident']):
    sentence_meta = metadata[metadata['sent_ident']==s]
    sentence_eeg = epochs_df[epochs_df['epoch'].isin(sentence_meta.index)]
    sentence_x = []
    sentence_y = []
    length = len(np.unique(sentence_eeg['epoch']))

    if length<5 or length>30:
        #skip sentences that are too long or too short
        continue

    for w in np.unique(sentence_eeg['epoch']):
        temp = np.zeros(1)
        e = sentence_eeg[sentence_eeg['epoch']==w]
        meta = sentence_meta.loc[w]
        epoch_reshape = np.array([[empty,   empty,   empty,      empty,      e['Fp1'],  empty,   e['Fp2'],  empty,   empty,      empty,     empty],
                                [empty,   empty,   empty,      e['AF7'],   e['AF3'],  e['AFz'],e['AF4'],  e['AF8'],empty,      empty,     empty],
                                [empty,   e['F7'], e['F5'],    e['F3'],    e['F1'],   e['Fz'], e['F2'],   e['F4'], e['F6'],    e['F8'],   empty],
                                [e['FT9'],e['FT7'],e['FC5'],   e['FC3'],   e['FC1'],  e['FCz'],e['FC2'],  e['FC4'],e['FC6'],   e['FT8'],  e['FT10']],
                                [empty,   e['T7'], e['C5'],    e['C3'],    e['C1'],   e['Cz'], e['C2'],   e['C4'], e['C6'],    e['T8'],   empty],
                                [e['TP9'],e['TP7'],e['CP5'],   e['CP3'],   e['CP1'],  e['CPz'],e['CP2'],  e['CP4'],e['CP6'],   e['TP8'],  e['TP10']],
                                [empty,   e['P7'], e['P5'],    e['P3'],    e['P1'],   e['Pz'], e['P2'],   e['P4'], e['P6'],    e['P8'],   empty],
                                [empty,   empty,   empty,      e['PO7'],   e['PO3'],  e['POz'],e['PO4'],  e['PO8'],empty,      empty,     empty],
                                [empty,   empty,   empty,      empty,      e['O1'],   e['Oz'], e['O2'],   empty,   empty,      empty,     empty]])
        sentence_x.append(epoch_reshape)
        pos = meta['pos']
        temp[temp == 0] = pos
        emb = temp
        sentence_y.append(emb)

    sentence_x.append(sent_end)
    sentence_y.append(pad)
    for i in range(31-length):
        sentence_x.append(sent_empty)
        sentence_y.append(pad)

    train_x.append(sentence_x)
    train_y.append(sentence_y)

In [33]:
train_x = np.array(train_x)
train_y = np.array(train_y)

In [34]:
train_x.shape

(161, 32, 9, 11, 276)

In [35]:
train_y.shape

(161, 32, 1)

In [36]:
pos_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-8),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

In [37]:
pos_model.fit(
    x = train_x,
    y =  train_y,
    batch_size = 32,
    epochs = 10,
    validation_split = 0.125
)

Epoch 1/10


ResourceExhaustedError: Graph execution error:

Detected at node StatefulPartitionedCall defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 619, in start

  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/usr/local/lib/python3.10/dist-packages/tornado/ioloop.py", line 685, in <lambda>

  File "/usr/local/lib/pyt